In [1]:
from config import DATABASE_URL
from src.utils.database_connection_utils import get_database_session

In [2]:
session, engine = get_database_session(DATABASE_URL)

In [3]:
sql_query = "select * from article where website_origin not in ('https://www.voalingala.com/', 'https://www.karismatv.cd')"

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = None
with engine.connect() as connection:
    data = pd.read_sql_query(sql=sql_query, con=connection)

In [13]:
data.columns

Index(['id', 'title', 'content', 'summary', 'posted_at', 'website_origin',
       'url', 'author', 'saved_at'],
      dtype='object')

In [37]:
data.website_origin.unique()

array(['https://forumdesas.net/', 'https://www.7sur7.cd',
       'https://laprosperiteonline.net', 'https://www.politico.cd',
       'https://www.actualite.cd', 'https://www.mediacongo.net',
       'https://actu30.cd', 'https://cas-info.ca',
       'https://lepotentiel.cd', 'https://forumdesas.net',
       'https://www.voalingala.com/', 'https://www.karismatv.cd'],
      dtype=object)

In [14]:
data = data.loc[~data.website_origin.isin(['https://www.voalingala.com/', 'https://www.karismatv.cd'])]

In [15]:
from pathlib import Path
DATA_PATH = Path.cwd().joinpath("data")

In [16]:
assert DATA_PATH.exists(), "the data path does not exist"

In [4]:
data.loc[:, "content"]

NameError: name 'data' is not defined

In [17]:
data.loc[:, ["content", "posted_at"]].to_csv(DATA_PATH.joinpath("corpus", "raw", 'drc-news-raws.csv'), index=False, header=False)

### Some verification about the content

In [9]:
random_index = np.random.randint(0, data.shape[0])
sample_doc = data.content[random_index]
sample_doc

"Le Tout Puissant Mazembe a battu le Football club Lubumbashi-Sport par un score de 3 buts à 1 au stade TP Mazembe de la commune de Kamalondo à Lubumbashi dans le Haut-Katanga, ce mercredi 17 février 2021.C'était en match de la phase retour de la Ligue Nationale de Football, Vodacom Ligue 1.C'est à la 16ème minute de jeu que les kamikazes ont ouvert le score sur un coup-franc tiré par le joueur Wandenga de Lubumbashi-Sport.Moins d'un quart d'heure après ce but de Lubumbashi-Sport, les corbeaux de Mazembe vont obtenir un penalty à la 30ème minute qui sera transformé par Christian Kouame après une faute de Kamikazes.Les deux équipes se sont séparées à la mi-temps par un but partout.De retour des vestiaires, Adam Bossu va alourdir le score pour le TP Mazembe à la 53ème minute. Cependant, à 4 minutes de la fin de la rencontre, le même attaquant va signer de la tête son doublé pour le but du break.Pour rappel, le Tout Puissant Mazembe avait battu le 26 novembre dernier le Football Club Lubu

How should I split the text , should I use the sentence or the word level to detect names in this entities?

I will try sentence and see how the french NER model from spacy will perform and then I will try more complicated approaches.

By the end of this work we should have a pipeline that takes the text and mask the name entities.

In [10]:
import re
from nltk.tokenize import sent_tokenize
from unicodedata import normalize

/Users/es.py/Library/Caches/pypoetry/virtualenvs/deep-learning-nlp-stuff/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Users/es.py/Library/Caches/pypoetry/virtualenvs/deep-learning-nlp-stuff/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted th

In [11]:
def remove_accents(input_str):
    nfkd_form = normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore').decode("utf-8")
    return only_ascii

def clean_text(text):
    cleaned_text = remove_accents(text)
    return str(cleaned_text)

In [12]:
clean_text(sample_doc)

"Le Tout Puissant Mazembe a battu le Football club Lubumbashi-Sport par un score de 3 buts a 1 au stade TP Mazembe de la commune de Kamalondo a Lubumbashi dans le Haut-Katanga, ce mercredi 17 fevrier 2021.C'etait en match de la phase retour de la Ligue Nationale de Football, Vodacom Ligue 1.C'est a la 16eme minute de jeu que les kamikazes ont ouvert le score sur un coup-franc tire par le joueur Wandenga de Lubumbashi-Sport.Moins d'un quart d'heure apres ce but de Lubumbashi-Sport, les corbeaux de Mazembe vont obtenir un penalty a la 30eme minute qui sera transforme par Christian Kouame apres une faute de Kamikazes.Les deux equipes se sont separees a la mi-temps par un but partout.De retour des vestiaires, Adam Bossu va alourdir le score pour le TP Mazembe a la 53eme minute. Cependant, a 4 minutes de la fin de la rencontre, le meme attaquant va signer de la tete son double pour le but du break.Pour rappel, le Tout Puissant Mazembe avait battu le 26 novembre dernier le Football Club Lubu

In [13]:
tokenized_document = sent_tokenize(clean_text(sample_doc), language="french")

In [14]:
tokenized_document

["Le Tout Puissant Mazembe a battu le Football club Lubumbashi-Sport par un score de 3 buts a 1 au stade TP Mazembe de la commune de Kamalondo a Lubumbashi dans le Haut-Katanga, ce mercredi 17 fevrier 2021.C'etait en match de la phase retour de la Ligue Nationale de Football, Vodacom Ligue 1.C'est a la 16eme minute de jeu que les kamikazes ont ouvert le score sur un coup-franc tire par le joueur Wandenga de Lubumbashi-Sport.Moins d'un quart d'heure apres ce but de Lubumbashi-Sport, les corbeaux de Mazembe vont obtenir un penalty a la 30eme minute qui sera transforme par Christian Kouame apres une faute de Kamikazes.Les deux equipes se sont separees a la mi-temps par un but partout.De retour des vestiaires, Adam Bossu va alourdir le score pour le TP Mazembe a la 53eme minute.",
 'Cependant, a 4 minutes de la fin de la rencontre, le meme attaquant va signer de la tete son double pour le but du break.Pour rappel, le Tout Puissant Mazembe avait battu le 26 novembre dernier le Football Club

In [15]:

import spacy

In [16]:
nlp = spacy.load("fr_core_news_sm")

In [17]:
from spacy import displacy

In [20]:
processed_sentence = nlp(tokenized_document[1])
processed_document = nlp(clean_text(sample_doc))

In [21]:
print(nlp.pipe_names)

['tok2vec', 'morphologizer', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']


In [22]:
tokenized_document[0]

"Le Tout Puissant Mazembe a battu le Football club Lubumbashi-Sport par un score de 3 buts a 1 au stade TP Mazembe de la commune de Kamalondo a Lubumbashi dans le Haut-Katanga, ce mercredi 17 fevrier 2021.C'etait en match de la phase retour de la Ligue Nationale de Football, Vodacom Ligue 1.C'est a la 16eme minute de jeu que les kamikazes ont ouvert le score sur un coup-franc tire par le joueur Wandenga de Lubumbashi-Sport.Moins d'un quart d'heure apres ce but de Lubumbashi-Sport, les corbeaux de Mazembe vont obtenir un penalty a la 30eme minute qui sera transforme par Christian Kouame apres une faute de Kamikazes.Les deux equipes se sont separees a la mi-temps par un but partout.De retour des vestiaires, Adam Bossu va alourdir le score pour le TP Mazembe a la 53eme minute."

In [23]:
displacy.render(processed_sentence, style="ent", jupyter=True)

In [24]:
displacy.render(processed_document, style="ent", jupyter=True)

The spacy model seems to be do well in detecting names, organization  ,but it can be improve to detec orgainization and more place.

In [23]:
{"meta": 
    {"source": "BBC News","via":"News API"},
 "_task_hash":142354664,
 "text":"ViewRay (VRAY +1.2%) inks an agreement with an affiliate of Fosun International Limited for the direct sale of stock and warrants yielding ~$59.1M in gross proceeds.Specifically, the company will sell",
 "_input_hash":1622806810}

{'meta': {'source': 'BBC News', 'via': 'News API'},
 '_task_hash': 142354664,
 'text': 'ViewRay (VRAY +1.2%) inks an agreement with an affiliate of Fosun International Limited for the direct sale of stock and warrants yielding ~$59.1M in gross proceeds.Specifically, the company will sell',
 '_input_hash': 1622806810}

Seems like that the spacy model is mislead by accent and `" `, so let us remove them from the text.

And improve the spacy annodation , if by saturaday I can'tafind anything I will need to continue with what I have and make it a baseline

###  Setting up Model Tracking Experiments

In [57]:
import mlflow

mlflow.set_tracking_uri("sqllite//:mlflow.db")
mlflow.set_experiment("congo-news-ner-experiments")

2022/05/24 12:00:37 INFO mlflow.tracking.fluent: Experiment with name 'congo-news-ner-experiments' does not exist. Creating a new experiment.


<Experiment: artifact_location='sqllite//:mlflow.db/1', experiment_id='1', lifecycle_stage='active', name='congo-news-ner-experiments', tags={}>

In [ ]:
with mlflow.run():
    mlflow.set_tag("corpus", "congo_news")

### NER Approaches

Here is the new approach I will use for the NER model.

- Select a random sample of dataset with 500 example , reannodate it in prodigy and save it .
- Use it to train a pre-train french NER model and save the new annotations.
- Re-annotate  the new results with correct and save it .
- Retrain the model with the new annotations and check the results if they make sense and use the new results as the new model.

Before training we need to build an new dataset which is loadable to pordigy and which can be used to train word embedding with fastext.

In [1]:
import pandas as pd
import numpy as np

In [2]:
from nltk.corpus.reader.util import concat
from gensim.corpora.csvcorpus import CsvCorpus
from gensim.utils import deaccent

/Users/es.py/Library/Caches/pypoetry/virtualenvs/deep-learning-nlp-stuff/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Users/es.py/Library/Caches/pypoetry/virtualenvs/deep-learning-nlp-stuff/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted th

In [22]:
from datetime import datetime

In [3]:
import logging
import csv
import itertools
from gensim import utils
logger = logging.getLogger(__name__)

In [4]:
import re
from nltk.tokenize import sent_tokenize
from unicodedata import normalize
from secrets import token_hex
import json
from pathlib import Path


In [54]:
import srsly

In [5]:
DATA_PATH = Path.cwd().joinpath("data")

In [80]:
 class DrcNewsCorpus(CsvCorpus):
    def __init__(self, fname, has_header=False):
        """

        Parameters
        ----------
        fname : str
            Path to corpus.
        labels : bool
            If True - ignore first column (class labels).

        """
        logger.info("loading corpus from %s", fname)
        self.fname = fname
        self.has_header = has_header
        self.dataset = pd.read_csv(fname, names=["content", "posted_at"])
        self.length = self.dataset.shape[0]

    def remove_accents(self, input_str):
        input_without_accent = deaccent(input_str)
        return input_without_accent

    def clean_text(self, text):
        cleaned_text = self.remove_accents(text)
        return str(cleaned_text)

    def tokenize_document(self, document):
        """given a document split the document into sentences and return a list of those sentence
    
        Args:
            document (_type_): _description_
    
        Returns:
            _type_: _description_
        """
        tokenized_document = sent_tokenize(document, language="french")
        return tokenized_document
    
    
    def to_prodigy_format(self, output_file_path, n_sample=None):
        """
        generate the json representation of the corpus and save it a a loadable version in csv
        Args:
            document_series (_type_): _description_

        Returns:
            _type_: _description_
        """
        with open(output_file_path, "w") as output_file:
            for item in self.__iter__(n_sample=n_sample):
                output_file.write(json.dumps(item, ensure_ascii=False) + "\n")
            

    def __iter__(self, n_sample=None):
        """Iterate over the corpus,  a line  at a time

        Yields
        ------
        list of (int, float)
            Document in BoW format.

        """
        if n_sample : 
            self.dataset = self.dataset.sample(n_sample)
        
        for _, line in self.dataset.fillna("").iterrows():
            cleaned_line = self.clean_text(line["content"])
            for sentence in self.tokenize_document(cleaned_line):
                posted_at = line["posted_at"] if line["posted_at"] else ""
                item = {"_task_hash": token_hex(6), "_input_hash": token_hex(6),  "text": sentence,  "posted_at": posted_at}
                yield item
    

In [81]:
drc_new_corpus_file = DATA_PATH.joinpath("corpus", "raw", "drc-news-raws.csv")

In [82]:
assert drc_new_corpus_file.exists()

In [83]:
drc_news_corpus = DrcNewsCorpus(drc_new_corpus_file, False)

In [84]:

sample_output_file = DATA_PATH.joinpath("corpus", "drc-news-raws-sample-600.jsonl")
drc_news_corpus.to_prodigy_format(sample_output_file, n_sample=600)

In [85]:

real_path = DATA_PATH.joinpath("corpus", "drc-news-raws-sample-600.jsonl")